In [ ]:
import pandas as pd

df = pd.read_csv("../../COALA_DS_DATA/COALA_DS_DATA/week3/data/train.csv")

df

In [ ]:
df['Fare'].value_counts()

In [ ]:
df[['Pclass', 'Fare']]

In [ ]:
df['Fare'] = df['Fare'].fillna(df.groupby('Pclass')['Fare'].transform("mean"))
df.head()

In [ ]:
df.loc[df['SibSp'] + df["Parch"] == 0, 'isAlone'] = 1
df.loc[df['SibSp'] + df["Parch"] > 0, 'isAlone'] = 0

df

In [ ]:
df.loc[df['Name'].str.contains("Mr\."), 'Name'] = 'Mr'
df.loc[df['Name'].str.contains("Mrs\."), 'Name'] = 'Mrs'
df.loc[df['Name'].str.contains("Miss\."), 'Name'] = 'Miss'
df[['Name', 'Age']]

In [ ]:
df['Name'] = df['Name'].map({
    'Mr': 0,
    'Mrs': 1,
    'Miss': 2
})

df['Name'] = df['Name'].fillna(3)

df

In [ ]:
df_test = pd.read_csv("../../COALA_DS_DATA/COALA_DS_DATA/week3/data/test.csv")

df_test

In [ ]:
df_test['Fare'].isnull().sum()

In [ ]:
df_test['Fare'] = df_test['Fare'].fillna(df_test.groupby('Pclass')['Fare'].transform("mean"))
df_test.head()

In [ ]:
df_test.loc[df_test['SibSp'] + df_test["Parch"] == 0, 'isAlone'] = 1
df_test.loc[df_test['SibSp'] + df_test["Parch"] > 0, 'isAlone'] = 0

df_test

In [ ]:
df['Embarked'] = df['Embarked'].fillna('S')
df_test['Embarked'] = df_test['Embarked'].fillna('S')

df.loc[df['Embarked'] == 'S', 'Embarked'] = 0
df.loc[df['Embarked'] == 'C', 'Embarked'] = 1
df.loc[df['Embarked'] == 'Q', 'Embarked'] = 2

df_test.loc[df_test['Embarked'] == 'S', 'Embarked'] = 0
df_test.loc[df_test['Embarked'] == 'C', 'Embarked'] = 1
df_test.loc[df_test['Embarked'] == 'Q', 'Embarked'] = 2


df.head()

In [ ]:
df.loc[df['Sex'] == 'male', 'Sex'] = 0
df.loc[df['Sex'] == 'female', 'Sex'] = 1

df_test.loc[df_test['Sex'] == 'male', 'Sex'] = 0
df_test.loc[df_test['Sex'] == 'female', 'Sex'] = 1

df.head()

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch']
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch']

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['Name'] = df_test['Name'].map({
    'Mr': 0,
    'Mrs': 1,
    'Miss': 2
})

df_test['Name'] = df_test['Name'].fillna(3)

In [ ]:
df_test['Age'] = df_test['Age'].fillna(df_test.groupby('Name')['Age'].transform('mean'))
df['Age'] = df['Age'].fillna(df.groupby('Name')['Age'].transform('mean'))

In [ ]:
x_train = df[['Name', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'isAlone']]
y_train = df['Survived'] 

x_train

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

print('training set accuracy:', tree.score(x_train, y_train))

In [ ]:
x_test = df_test[['Name', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'isAlone']]
prediction = tree.predict(x_test) 
prediction

In [ ]:
submit = pd.DataFrame({
    'PassengerId': df_test['PassengerId'],
    'Survived': prediction
})

submit.to_csv('submit.csv', index=False)

In [ ]:
my_prediction = pd.read_csv('submit.csv')
my_prediction.head()

# Random Forest
---
Decision Tree는 일부 데이터에 대해 과적합(Overfitting) 현상으로 실제 데이터 예측 성공 확률이 낮아질 수 있음.

그것을 보완하기 위해 Random Forest는 여러 Decision Tree의 결과를 종합, 평균을 계산(Ensemble)하여 예측하는 모델


### Stage1

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

print('training set accuracy : ', tree.score(x_train, y_train))

In [ ]:
tree1 = DecisionTreeClassifier()
tree1.fit(x_train, y_train)

print('training set accuracy : ', tree1.score(x_train, y_train))

In [ ]:
tree2 = DecisionTreeClassifier()
tree2.fit(x_train, y_train)

print('training set accuracy : ', tree2.score(x_train, y_train))

In [ ]:
# 위의 3개의 트리는 모두 다른 트리다.
# 문제나 데이터가 복잡해지면 정확도는 달라짐
# 연결해보기 (Ensemble)

# Validation_Set -> 훈련에서 제외시킬 테스트(학습된 모델의 중간 성능)용 데이터
x_valid = x_train[:100]
y_valid = y_train[:100]

x_train = x_train[100:]
y_train = y_train[100:]

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

print('training set accuracy : ', tree.score(x_train, y_train))
print('validation set accuracy : ', tree.score(x_valid, y_valid))

tree1 = DecisionTreeClassifier()
tree1.fit(x_train, y_train)

print('training set accuracy : ', tree1.score(x_train, y_train))
print('validation set accuracy : ', tree1.score(x_valid, y_valid))

tree2 = DecisionTreeClassifier()
tree2.fit(x_train, y_train)

print('training set accuracy : ', tree2.score(x_train, y_train))
print('validation set accuracy : ', tree2.score(x_valid, y_valid))

# validation accuracy 는 매번 달라지는 것을 보면, 서로 다른 트리임을 알 수 있다.
# val_accuracy 가 더 합리적인 성능 지표

In [ ]:
# 여러개의 Decision Tree Ensemble 하기
prediction1 = tree.predict(df_test[['Name', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'isAlone']])
prediction2 = tree1.predict(df_test[['Name', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'isAlone']])
prediction3 = tree2.predict(df_test[['Name', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'isAlone']])

ensemble = (prediction1 + prediction2 + prediction3) / 3
ensemble[ensemble > 0.5] = 1
ensemble[ensemble <= 0.5] = 0

# 평균 내는 작업으로 인해 survived 데이터가 정수에서 소수로 바뀜
# 다시 정수로 변환
ensemble = ensemble.astype(int)

ensemble

In [ ]:
submit = pd.DataFrame({
    'PassengerId' : df_test['PassengerId'],
    'Survived' : ensemble
})

submit.to_csv("submit_ensemble.csv", index = False)

## Ch2

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators = 3000)
rfc.fit(x_train, y_train)

print('training set accuracy : ', rfc.score(x_train, y_train))

In [ ]:
print('validation set accuracy : ', rfc.score(x_valid, y_valid))

In [ ]:
prediction2 = rfc.predict(x_test)
print(prediction)

submit = pd.DataFrame({
    'PassengerId' : df_test["PassengerId"],
    'Survived' : prediction
})

submit.to_csv('submit_rfc.csv', index = False)

## HW1

In [ ]:
x_all = df[['Name', 'Sex', 'Age', 'Fare', 'FamilySize', 'Embarked', 'isAlone']]
y_all = df['Survived']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc2 = RandomForestClassifier(n_estimators = 3000)
rfc2.fit(x_all, y_all)

print('training set accuracy : ', rfc2.score(x_all, y_all))

In [ ]:
prediction2 = rfc2.predict(x_test)
print(prediction2)

submit = pd.DataFrame({
    'PassengerId' : df_test["PassengerId"],
    'Survived' : prediction
})

submit.to_csv('submit_rfc2.csv', index = False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc3 = RandomForestClassifier(n_estimators = 100, max_depth = 25)
rfc3.fit(x_all, y_all)

print("training set accuracy :", rfc3.score(x_all, y_all))

In [ ]:
prediction3 = rfc3.predict(x_test)
print(prediction3)

submit = pd.DataFrame({
    'PassengerId' : df_test["PassengerId"],
    'Survived' : prediction
})

submit.to_csv('submit_rfc3.csv', index = False)